<a href="https://colab.research.google.com/github/anniebritton/Eco-Drought-South-Dakota/blob/main/NDVI_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Colab Set Up**

In [ ]:
# installs and import libraries
!pip install geopandas 
!pip install matplotlib
!pip install scikit-learn
!pip install lazypredict
!pip install shap

import pandas as pd
import numpy as np

import lazypredict
import shap
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
from matplotlib import ticker
plt.rcParams["figure.figsize"] = (20,3)

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Data**

In [74]:
# import the dataset only for the full range of NDVI data 2000 on
df_ndvi = pd.read_csv('/content/drive/MyDrive/School/M.S./Courses/Capstone/Colab/Data/CSVs/drought_NDVI_range_V2.csv')
df_ndvi['date'] = pd.to_datetime(df_ndvi['date']).dt.strftime('%Y-%m-%d')
df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
df_ndvi = df_ndvi.set_index('date')
# df_ndvi = df_ndvi.drop(['smam', 'smpm', 'PP_NDVI'], axis=1)

In [75]:
# import and create a list of pentads that the drought data is based on. 
# This will be used to calculate anomalies and average so that all of the 
# data variables match temporally
pentads = pd.read_csv('/content/drive/MyDrive/School/M.S./Courses/Capstone/Colab/Data/CSVs/drought_variable_pentads.csv')
pentads = pentads[1470:]
pentads = pentads['date'].tolist()

**Calculate Anomalies**

In [76]:
# Function that will calculate the daily mean for each variable and then 
# subsequently calculate the anomaly for each variable/day
def calculate_anomaly(df, value_col):
    
    # Group the data by day of the year and calculate the average for each day of the year
    df_daily_grouping = df.groupby(df.index.dayofyear).mean()

    # Create a dictionary mapping day of year to average value
    day_of_year_to_mean = df_daily_grouping[value_col].to_dict()

    # Map the day of year to the average value for that day of year
    df['day_of_year'] = df.index.dayofyear
    df[f'day_of_year_{value_col}_mean'] = df['day_of_year'].map(day_of_year_to_mean)

    # Calculate the daily anomaly as the difference between the original value and the average value for that day of year
    df[f'{value_col}_anomaly'] = df[value_col] - df[f'day_of_year_{value_col}_mean']

# NOTE - I am doing this before resampling we want to look at each day of the year here.
# If we did this after resampling, because of the way the pentads work, we would have
# less data to compare year to year.

# Apply the function to each column of the dataframe
for col in df_ndvi.columns:
    if col != 'date':
        calculate_anomaly(df_ndvi, col)

In [77]:
# Reduce the dataframe so that it only contains the anomaly data
df_anom = df_ndvi[df_ndvi.columns[df_ndvi.columns.str.endswith('_anomaly')]]

# Move the NDVI column to the first position in the dataframe
df_anom.insert(0, 'NDVI_anomaly', df_anom.pop('NDVI_anomaly'))

**Resample the Data to match the Pentads from the Drought Indexes, and Normalize**

In [103]:
# THIS NARROWS THE DATA DOWN TO PENTADS - only have data every 5 days
# create an empty list to store the result dataframes
results = []

# loop through each column in the dataframe
for col in df_anom.columns:
    if col == 'NDVI_anomaly':
        result = pd.DataFrame({'date': pentads, f'{col}_roll': df_anom[col].rolling(window=30, min_periods=1).mean().loc[pentads]})
    else:
        result = pd.DataFrame({'date': pentads, f'{col}_roll': df_anom[col].rolling(window=30, min_periods=1).mean().loc[pentads]})
    result = result.set_index('date')
    results.append(result)

# concatenate the result dataframes into a single dataframe
anom_result = pd.concat(results, axis=1)

# remove the first row since that row is not based on a 5-day average
anom_result = anom_result[1:]

# normalize the data
normalized_df = (anom_result-anom_result.mean())/anom_result.std()

**Visualize**

In [ ]:
b = normalized_df['pdsi_anomaly_roll']
c = normalized_df['NDVI_anomaly_roll']
fig, ax = plt.subplots(1,1, )
ax.plot(b, label='PDSI')
ax.plot(c, label='NDVI')
ax.xaxis.set_major_locator(ticker.LinearLocator(numticks = 15))
ax.legend()
plt.title("NDVI 30 Day Rolling Mean")
fig.show()

**Prelim ML Tests**

In [ ]:
# try a random forest classifier
from sklearn.ensemble import RandomForestClassifier

# add in a boolean drought column (1 for drought, 0 for no drought)
normalized_df['drought_bool'] = np.where(normalized_df["pdsi_anomaly_roll"] >= 0, 0, 1)

X = normalized_df.iloc[:,0:15].values
Y = normalized_df.iloc[:,15:16].values.ravel()

clf = RandomForestClassifier(n_estimators=100)

# SUBSET YOUR X AND Y INTO TRAIN/TEST SETS; for instance, take first 80% of the rows as training, last 20% as test
X_train = normalized_df.iloc[0:1272, 0:15].values
Y_train = normalized_df.iloc[0:1272, 15:16].values.ravel()
X_test = normalized_df.iloc[1272:1590, 0:15].values
Y_test = normalized_df.iloc[1272:1590, 15:16].values

clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_test)

# compare Y_predicted with Y_test
clf.score(X_test, Y_test)

In [ ]:
# try a decision tree regressor
from sklearn import tree

X = normalized_df.iloc[:,1:15].values
Y = normalized_df.iloc[:,0:1].values.ravel()

clf = tree.DecisionTreeRegressor()

X_train = normalized_df.iloc[0:1272, 1:15].values
Y_train = normalized_df.iloc[0:1272, 0:1].values.ravel()
X_test = normalized_df.iloc[1272:1590, 1:15].values
Y_test = normalized_df.iloc[1272:1590, 0:1].values

clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_test)

# compare Y_predicted with Y_test
clf.score(X_test, Y_test)

In [ ]:
# try support vector regression
from sklearn import svm

X = normalized_df.iloc[:,1:15].values
Y = normalized_df.iloc[:,0:1].values.ravel()

regr = svm.SVR()

X_train = normalized_df.iloc[0:1272, 1:15].values
Y_train = normalized_df.iloc[0:1272, 0:1].values.ravel()
X_test = normalized_df.iloc[1272:1590, 1:15].values
Y_test = normalized_df.iloc[1272:1590, 0:1].values

regr.fit(X_train, Y_train)
Y_predicted = regr.predict(X_test)

# compare Y_predicted with Y_test
regr.score(X_test, Y_test)

**Trying Out LazyPredict**

In [ ]:
# install and import
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

In [ ]:
# Classification
# add in a boolean drought column (1 for drought, 0 for no drought)
normalized_df['drought_bool'] = np.where(normalized_df["pdsi_anomaly_roll"] >= 0, 0, 1)

X_train = normalized_df.iloc[0:1272, 0:15].values
y_train = normalized_df.iloc[0:1272, 15:16].values.ravel()

X_test = normalized_df.iloc[1272:1590, 0:15].values
y_test = normalized_df.iloc[1272:1590, 15:16].values

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
cmodel,cprediction = clf.fit(X_train, X_test, y_train, y_test)

cmodel

In [ ]:
# Regression
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

X_train = normalized_df.iloc[0:1272, 0:15].values
y_train = normalized_df.iloc[0:1272, 15:16].values.ravel()

X_test = normalized_df.iloc[1272:1590, 0:15].values
y_test = normalized_df.iloc[1272:1590, 15:16].values

rmodel, rprediction = reg.fit(X_train, X_test, y_train, y_test)

rmodel

**K-fold Cross Validation for Lazy Predict Regression**

In [148]:
from sklearn.model_selection import KFold

def k_fold_lp(data, target, k=5):
    # Create a KFold object with k folds
    kf = KFold(n_splits=k, shuffle = True, random_state=42)
    # Create an empty df to store scores for each fold
    scores = pd.DataFrame()
    # Loop over each fold
    for train_idx, test_idx in kf.split(data):
        # Split the data into train and test sets for this fold
        X_train, X_test = data[train_idx], data[test_idx]
        y_train, y_test = target[train_idx], target[test_idx]
        # Create a LazyRegressor model
        reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
        # Fit the model on the train data and make predictions on the test data
        models, predictions = reg.fit(X_train, X_test, y_train, y_test)
        # Append the score to the list of scores for this fold
        scores = scores.append(predictions)
    # Calculate the mean of all scores across all folds
    return scores

In [ ]:
X = normalized_df.iloc[:, 1:15].values
y = normalized_df.iloc[:, 0:1].values.ravel()

mean_score = k_fold_lp(X, y)
mean_score = mean_score.sort_values(by = "Adjusted R-Squared", ascending = False)

In [161]:
mean_score.groupby("Model").mean().sort_values(by = "Adjusted R-Squared", ascending = False)

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GaussianProcessRegressor,0.93,0.93,0.26,0.26
ExtraTreesRegressor,0.93,0.93,0.26,0.69
LGBMRegressor,0.89,0.89,0.32,0.27
HistGradientBoostingRegressor,0.89,0.89,0.32,1.05
XGBRegressor,0.89,0.89,0.32,0.60
RandomForestRegressor,0.87,0.88,0.34,1.76
BaggingRegressor,0.86,0.86,0.37,0.18
KNeighborsRegressor,0.82,0.82,0.42,0.04
GradientBoostingRegressor,0.78,0.79,0.46,0.86


**K-Fold on Individual Models**

In [157]:
from sklearn.utils import shuffle
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score

X = normalized_df.iloc[:, 1:15].values
y = normalized_df.iloc[:, 0:1].values.ravel()
#X, y = shuffle(X, y, random_state=42)

# Define your XGBoost model
xgb_model = xgb.XGBRegressor()

# Define the number of folds for cross validation
num_folds = 5

# Define the k-fold cross validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Run k-fold cross validation on the XGBoost model
results = cross_val_score(xgb_model, X, y, cv=kfold, scoring='r2')

# Print the mean and standard deviation of the results
print("Results:", results)
print("Mean Result:", results.mean())

Results: [0.87942551 0.91035714 0.85577327 0.92265321 0.8955487 ]
Mean Result: 0.8927515641699184


In [160]:
from sklearn.utils import shuffle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import KFold, cross_val_score

X = normalized_df.iloc[:, 1:15].values
y = normalized_df.iloc[:, 0:1].values.ravel()
#X, y = shuffle(X, y, random_state=42)

# Define your model
gpr_model = GaussianProcessRegressor()

# Define the number of folds for cross validation
num_folds = 5

# Define the k-fold cross validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Run k-fold cross validation on the model
results = cross_val_score(gpr_model, X, y, cv=kfold, scoring='r2')

# Print the mean and standard deviation of the results
print("Results:", results)
print("Mean Result:", results.mean())

Results: [0.90848108 0.93312081 0.95463627 0.95008162 0.91378848]
Mean Result: 0.9320216516154775


**Trying Different Cross Val Strategies**

In [ ]:
# Shuffle Split

from sklearn.model_selection import ShuffleSplit
X = np.arange(10)
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_index, test_index in ss.split(X):
    print("%s %s" % (train_index, test_index))

**Hyperparameter Optimization**

In [114]:
from sklearn.model_selection import RandomizedSearchCV

X = normalized_df.iloc[:, 1:15].values
y = normalized_df.iloc[:, 0:1].values.ravel()

# define the model
model = XGBRegressor()

# define the hyperparameter grid
hyperparameters = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]
}
#     'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5],
#     'colsample_bytree': [0.3, 0.4, 0.5, 0.7, 0.9],
#     'gamma': [0, 0.1, 0.2, 0.3, 0.4],
#     'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]
# }

# define the random search object
random_search = RandomizedSearchCV(
    estimator=model,         # specify the model to use
    param_distributions=hyperparameters,  # specify the hyperparameter grid to search over
    n_iter=50,               # specify the number of combinations of hyperparameters to try
    cv=5,                    # specify the number of cross-validation folds to use
    n_jobs=-1,               # specify the number of CPU cores to use (-1 means use all available cores)
    scoring='neg_mean_squared_error'   # specify the metric to optimize for
)

# perform the random search
random_search.fit(X, y)

# print the best hyperparameters
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

{'n_estimators': 100, 'max_depth': 6}
-0.5123562568811239
